In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
import jsonref
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
mcp_server_name = os.getenv("MCP_SERVER_NAME")

In [2]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

In [3]:
openai_client = project_client.get_openai_client()

In [5]:
with open("./weather_openapi.json", "r") as f:
        openapi_weather = jsonref.loads(f.read())

# Initialize agent OpenApi tool using the read in OpenAPI spec
weather_tool = {
        "type": "openapi",
        "openapi":{
            "name": "weather",
            "spec": openapi_weather,
            "auth": {
                "type": "anonymous"
            },
        }
}

In [6]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == mcp_server_name:
        connection_id = connection.id
        break

print(f"The MCP Server Connection ID is: {connection_id}")

The MCP Server Connection ID is: /subscriptions/3c1227d7-cdd2-4efc-985f-e304cc9273ea/resourceGroups/foundarydemorg/providers/Microsoft.CognitiveServices/accounts/demoprojectfractal/projects/demo-proj/connections/mslearn-mcp-server


In [7]:
mcp_tool = MCPTool(
    server_label = "microsoft_learn_server",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="never",
    project_connection_id=connection_id
)

In [8]:
agent_name = "multi-tool-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful assistant that can use multiple tools to answer user queries.",
        tools=[weather_tool, mcp_tool],
    )
)

print(f"Created agent: {agent.name} with ID: {agent.id}")

Created agent: multi-tool-agent with ID: multi-tool-agent:1


In [9]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

Created conversation with id: conv_6ed11d22dd02b13800eSIE5COrHxOKzlP4PDiQBFx1hIV6rZGU


In [10]:
user_query = "Let me know the weather in Seattle and also find me a Microsoft Learn module on Microsoft Foundry."

In [11]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body = {
        "agent": {
            "name": agent_name,
            "type": "agent_reference"
        }
    },
    input = user_query
)

print(f"Agent response: {response.output_text}")

Agent response: ### Current Weather in Seattle

- **Temperature:** 2°C (35°F)
- **Feels Like:** 1°C (34°F)
- **Weather:** Fog
- **Humidity:** 100%
- **Wind:** ESE at 4 kmph (2 mph)
- **Pressure:** 1022 mb
- **Visibility:** 0 km

### Microsoft Learn Module on Microsoft Foundry

You can explore [Microsoft Foundry](https://learn.microsoft.com/en-us/azure/ai-foundry/what-is-azure-ai-foundry?view=foundry-classic) which is designed for developers to build generative AI applications on an enterprise-grade platform. It offers tools and models to explore, build, test, and deploy AI applications, emphasizing responsible AI practices. Key features include self-serve capabilities, collaboration tools, and integration with Azure resources.

For more detailed guidance, visit the [Microsoft Foundry Quickstart](https://learn.microsoft.com/en-us/azure/ai-foundry/quickstarts/get-started-code?view=foundry-classic) to get started with creating projects and deploying models.
